<img src="Images/HSP2.png" />
This Jupyter Notebook Copyright 2017 by RESPEC, INC.  All rights reserved.

$\textbf{HSP}^{\textbf{2}}\ \text{and}\ \textbf{HSP2}\ $ Copyright 2017 by RESPEC INC. and released under this [License](LegalInformation/License.txt)

### Required Python imports  and setup

In [2]:
import os
import site
site.addsitedir(os.getcwd().rsplit('\\',1)[0] + '\\')  # adds your path to the HSP2 software.

hdfname = 'TutorialData/tutorial.h5'

import shutil
import numpy as np
import pandas as pd
pd.options.display.max_rows    = 18
pd.options.display.max_columns = 10
pd.options.display.float_format = '{:.2f}'.format  # display 2 digits after the decimal point

import HSP2
import HSP2tools

HSP2tools.reset_tutorial()    # make a new copy of the tutorial's data
HSP2tools.versions()          # display version information below

import matplotlib.pyplot as m_plt
%matplotlib inline
# %matplotlib notebook

## Run $\textbf{HSP}^\textbf{2}$

In [3]:
HSP2.run(hdfname, saveall=True)

C:\Users\Jason.Love\AppData\Local\Continuum\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: FutureWarning: get_store is deprecated and be removed in a future version
HDFStore(path, **kwargs) is the replacement
  interactivity=interactivity, compiler=compiler, result=result)


2017-11-16 15:56:12.39   Run Started for file TutorialData/tutorial.h5
2017-11-16 15:56:12.44     Start 1976-01-01 00:00    Stop 1976-12-31 23:59
2017-11-16 15:56:12.95   Finished setup
2017-11-16 15:56:12.95      PERLND P001   DELT=60
2017-11-16 15:56:33.75      RCHRES R001   DELT=60
2017-11-16 15:56:49.08      RCHRES R002   DELT=60
2017-11-16 15:56:54.29      RCHRES R003   DELT=60
2017-11-16 15:56:54.33      RCHRES R004   DELT=60
2017-11-16 15:56:54.39      IMPLND I001   DELT=60
2017-11-16 15:56:56.06      RCHRES R005   DELT=60
2017-11-16 15:56:56.12   Run completed


In [4]:
# review simulated flow from master hdf5 file for Reach 1 prior to commencing calibration
tsMaster = pd.read_hdf(hdfname, '/RESULTS/RCHRES_R001/HYDR')
tsMaster.head()

,DEP,IVOL,O1,O2,OVOL1,...,SAREA,TAU,USTAR,VOL,VOLEV
1976-01-01 00:00:00,9.95,0.02,0.00,0.00,0.00,...,6.03,0.00,0.00,30.02,0.00
1976-01-01 01:00:00,9.96,0.02,0.00,0.00,0.00,...,6.03,0.00,0.00,30.04,0.00
1976-01-01 02:00:00,9.96,0.02,0.00,0.00,0.00,...,6.04,0.00,0.00,30.06,0.00
1976-01-01 03:00:00,9.96,0.02,0.00,0.00,0.00,...,6.04,0.00,0.00,30.09,0.00
1976-01-01 04:00:00,9.97,0.02,0.00,0.00,0.00,...,6.04,0.00,0.00,30.11,0.00


In [5]:
# Summaize Hourly Flow
descMaster = tsMaster.RO.describe()
     
# Save For Later
index = descMaster.index
columns = ['Master','Run1', 'Run2']
dfStats = pd.DataFrame(index = index, columns=columns)
dfStats.Master = tsMaster.RO.describe()

dfStats.Master

count   8784.00
mean       3.33
std        4.35
min        0.00
25%        0.00
50%        0.79
75%        6.64
max       21.67
Name: Master, dtype: float64

## There are Multiple ways we Can Change Parameters

###  Using HSP2Tools.Fetch and then HSP2Tools.replace

In [6]:
replaceinfo, df = HSP2tools.fetch(hdfname, 'PERLND', 'PWATER', 'PARAMETERS')
replaceinfo, df

(('TutorialData/tutorial.h5', 'PERLND', 'PWATER'),
       AGWETP  AGWRC  BASETP  CEPSC  DEEPFR  ...   PETMIN  SLSUR  UZSN  FZG  \
 P001    0.08   0.98    0.00   0.00    0.10  ...    35.00   0.05  0.01 1.00   
 
       FZGL  
 P001  0.10  
 
 [1 rows x 22 columns])

In [7]:
# Check Value of LZSN
df.LZSN

P001   8.00
Name: LZSN, dtype: float64

In [8]:
# Change and Replace using HSP2Tools.replace?? 
df.LZSN = 12
HSP2tools.replace(replaceinfo, df)

### Use Pandas to Read and Write

In [9]:
datapath = '/PERLND/PWATER/PARAMETERS'
df2 = pd.read_hdf(hdfname, datapath)
df2.T

,P001
AGWETP,0.08
AGWRC,0.98
BASETP,0.00
CEPSC,0.00
DEEPFR,0.10
FOREST,0.01
INFEXP,2.00
INFILD,2.00
INFILT,0.15
...,...


In [10]:
# Use Pandas Read and Write - Write back to Origninal
df2.LZSN = 8
df2.to_hdf(hdfname, '/PERLND/PWATER/PARAMETERS', data_columns=True, format='table')

In [11]:
# Verify Changed
df2 = pd.read_hdf(hdfname, datapath)
df2.LZSN

P001    8
Name: LZSN, dtype: int64

### Use HSPTools.csvReader to Update Paramters

In [25]:
pd.read_csv('TutorialData/PERLND.csv')

,SEGMENT,INFILT,LZSN
0,P001,0.30,12


In [29]:
# Use CSV to Update the HDF5 File
HSP2tools.csvReader(hdfname, 'TutorialData/PERLND.csv', 'PERLND', 'PWATER')
var = ['INFILT','LZSN']
pd.read_hdf(hdfname, datapath)[var]

,INFILT,LZSN
P001,0.30,12


### Use HSPTools.DoE (Design of Experiment) function to Update Paramters

In [14]:
# How Can we Cange Multiple Parameters and Track Changes
# Using the HSP2.DoE approach allows us to create new folders within HDF5 file to document 
# changes from base and store new results
data = [
 ['1', 'PERLND', 'P001', 'PWATER', 'INFILT',   0.075],
 ['1', 'PERLND', 'P001', 'PWATER', 'LZSN',         4],
 ['2', 'PERLND', 'P001', 'PWATER', 'INFILT',    0.30],
 ['2', 'PERLND', 'P001', 'PWATER', 'LZSN',      12.0]]

doe = pd.DataFrame(data, columns=['Run', 'Operation', 'Segment', 'Module', 'Parameter', 'Value'])
doe

,Run,Operation,Segment,Module,Parameter,Value
0,1,PERLND,P001,PWATER,INFILT,0.07
1,1,PERLND,P001,PWATER,LZSN,4.00
2,2,PERLND,P001,PWATER,INFILT,0.30
3,2,PERLND,P001,PWATER,LZSN,12.00


In [15]:
HSP2.run_DoE(hdfname, 'Sensitivity_LZSN_INFILT', doe, saveall=True)

2017-11-16 15:56:57.97   HSP2 Started for file TutorialData/tutorial.h5
2017-11-16 15:56:57.97      Design of Experiment HDF5 directory is Sensitivity_LZSN_INFILT
2017-11-16 15:56:57.98     Start 1976-01-01 00:00    Stop 1976-12-31 23:59
2017-11-16 15:56:58.70   Finished setup
2017-11-16 15:56:58.73      Starting RUN 1
2017-11-16 15:56:58.73      PERLND P001   DELT=60
2017-11-16 15:56:58.85      RCHRES R001   DELT=60
SIMPATH is Sensitivity_LZSN_INFILT/RESULTS/RUN1/PERLND_P001/PWATER
2017-11-16 15:56:58.90      RCHRES R002   DELT=60
SIMPATH is Sensitivity_LZSN_INFILT/RESULTS/RUN1/RCHRES_R001/HYDR
2017-11-16 15:56:58.95      RCHRES R003   DELT=60
SIMPATH is Sensitivity_LZSN_INFILT/RESULTS/RUN1/RCHRES_R001/HYDR
2017-11-16 15:56:58.98      RCHRES R004   DELT=60
SIMPATH is Sensitivity_LZSN_INFILT/RESULTS/RUN1/RCHRES_R002/HYDR
SIMPATH is Sensitivity_LZSN_INFILT/RESULTS/RUN1/RCHRES_R003/HYDR
2017-11-16 15:56:59.04      RCHRES R005   DELT=60
SIMPATH is Sensitivity_LZSN_INFILT/RESULTS/RUN1/IMPL

In [16]:
# Acquire and Calculate Stats on Run 1 & Run 2 and Compare to Master
tsRun_1  = pd.read_hdf(hdfname, 'Sensitivity_LZSN_INFILT/RESULTS/RUN1/RCHRES_R001/HYDR')
tsRun_2  = pd.read_hdf(hdfname, 'Sensitivity_LZSN_INFILT/RESULTS/RUN2/RCHRES_R001/HYDR')

dfStats.Run1 = tsRun_1.RO.describe()
dfStats.Run2 = tsRun_2.RO.describe()
dfStats

,Master,Run1,Run2
count,8784.00,8784.00,8784.00
mean,3.33,5.08,2.44
std,4.35,8.66,3.28
min,0.00,0.00,0.00
25%,0.00,0.00,0.00
50%,0.79,1.74,0.43
75%,6.64,7.09,4.49
max,21.67,308.45,14.49


In [17]:
m_plt.figure(figsize=(16,8))
m_plt.plot('RO', 'b--', data=tsMaster,   label='Master')
m_plt.plot('RO', 'r',   data=tsRun_1,    label='Run1')
m_plt.plot('RO', 'g-',   data=tsRun_2,  label='Run2')
m_plt.title('Flow at Reach 1')
m_plt.ylabel('Hourly Flow {CFS}')
m_plt.legend(loc='best') 

<IPython.core.display.Javascript object>

In [18]:
# Resample for Monthly Flow
m_plt.figure(figsize=(16,8))
m_plt.plot('RO', 'b--', data=tsMaster.resample('M').mean(),   label='Master')
m_plt.plot('RO', 'r',   data=tsRun_1.resample('M').mean(),    label='Run1')
m_plt.plot('RO', 'g-',   data=tsRun_2.resample('M').mean(),  label='Run2')
m_plt.title('Flow at Reach 1')
m_plt.ylabel('Monthly Flow {CFS}')
m_plt.legend(loc='best') 

<IPython.core.display.Javascript object>

In [19]:
N = 20   # number of runs - generally large such as 1000 or 10,000

data = []
for r in range(1, N+1):
    run = str(r+10)
    data.append([run,'PERLND','P001','PWATER','LZSN', 1 + float(np.random.rand(1)) * 15])
   
doe = pd.DataFrame(data, columns=['Run', 'Operation', 'Segment', 'Module', 'Parameter', 'Value'])
doe

,Run,Operation,Segment,Module,Parameter,Value
0,11,PERLND,P001,PWATER,LZSN,9.72
1,12,PERLND,P001,PWATER,LZSN,5.99
2,13,PERLND,P001,PWATER,LZSN,4.19
3,14,PERLND,P001,PWATER,LZSN,14.10
4,15,PERLND,P001,PWATER,LZSN,13.79
5,16,PERLND,P001,PWATER,LZSN,5.40
6,17,PERLND,P001,PWATER,LZSN,8.82
7,18,PERLND,P001,PWATER,LZSN,14.70
8,19,PERLND,P001,PWATER,LZSN,1.79
...,...,...,...,...,...,...


In [20]:
HSP2.run_DoE(hdfname, 'P001_LZSN', doe)

2017-11-16 15:57:00.14   HSP2 Started for file TutorialData/tutorial.h5
2017-11-16 15:57:00.14      Design of Experiment HDF5 directory is P001_LZSN
2017-11-16 15:57:00.16     Start 1976-01-01 00:00    Stop 1976-12-31 23:59
2017-11-16 15:57:00.79   Finished setup
2017-11-16 15:57:00.81      Starting RUN 11
2017-11-16 15:57:00.81      PERLND P001   DELT=60
2017-11-16 15:57:00.92      RCHRES R001   DELT=60
SIMPATH is P001_LZSN/RESULTS/RUN11/PERLND_P001/PWATER
2017-11-16 15:57:00.96      RCHRES R002   DELT=60
SIMPATH is P001_LZSN/RESULTS/RUN11/RCHRES_R001/HYDR
2017-11-16 15:57:01      RCHRES R003   DELT=60
SIMPATH is P001_LZSN/RESULTS/RUN11/RCHRES_R001/HYDR
2017-11-16 15:57:01.05      RCHRES R004   DELT=60
SIMPATH is P001_LZSN/RESULTS/RUN11/RCHRES_R002/HYDR
SIMPATH is P001_LZSN/RESULTS/RUN11/RCHRES_R003/HYDR
2017-11-16 15:57:01.09      RCHRES R005   DELT=60
SIMPATH is P001_LZSN/RESULTS/RUN11/IMPLND_I001/IWATER
NEED NON SIMPATH DATA /RESULTS/IMPLND_I001/IWATER
SIMPATH is P001_LZSN/RESULTS/

In [21]:
keys = ['P001_LZSN/RESULTS/RUN' + str(k+10) + '/RCHRES_R001/HYDR' for k in range(1,N+1)]
keys

['P001_LZSN/RESULTS/RUN11/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN12/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN13/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN14/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN15/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN16/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN17/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN18/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN19/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN20/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN21/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN22/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN23/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN24/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN25/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN26/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN27/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN28/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN29/RCHRES_R001/HYDR',
 'P001_LZSN/RESULTS/RUN30/RCHRES_R001/HYDR']

In [22]:
ts = pd.DataFrame()
for k in keys:
    colname = k[18:23]
    df = pd.read_hdf(hdfname, k)
    ts[colname] = df.ROVOL
    
ts.plot(label = 'simulated volume (ac-ft)',figsize = (18,6))

<IPython.core.display.Javascript object>

In [23]:
df = ts.describe()
df

,RUN11,RUN12,RUN13,RUN14,RUN15,...,RUN26,RUN27,RUN28,RUN29,RUN30
count,8784.00,8784.00,8784.00,8784.00,8784.00,...,8784.00,8784.00,8784.00,8784.00,8784.00
mean,0.27,0.40,0.48,0.15,0.15,...,0.29,0.25,0.34,0.18,0.15
std,0.37,0.53,0.62,0.20,0.21,...,0.40,0.34,0.45,0.24,0.20
min,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00
50%,0.05,0.12,0.23,0.03,0.03,...,0.05,0.04,0.07,0.03,0.03
75%,0.54,0.72,0.81,0.25,0.26,...,0.57,0.49,0.62,0.31,0.25
max,1.84,2.49,2.77,0.70,0.73,...,2.00,1.72,2.21,0.90,0.70
